In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:

!pip -qqq install pydub
!pip -qqq install tensorboard
!pip -qqq install visualkeras
!pip -qqq install --upgrade keras
!pip -qqq install --upgrade keras tensorflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.4/997.4 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 108.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 117.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-text 2.18.1 requires tensorflow<2.19,>=2.18.0, but you have tensorflow 2.19.0 which is incompatible.
tensorflow-decision-forests 1.11.0 requires tensorflow==2.18.0, but you have tensorflow 2.19.0 which is incompatible.
tf-keras 2.18.0 requires tensorflow<2.19,>=2.18, but you have tensorflow 2.19.0 which is incompatible.


In [3]:
import os
import math
import numpy as np
import librosa
import json
import pandas as pd
from termcolor import colored
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
import visualkeras
from keras.utils import plot_model
from datetime import datetime
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn import svm
from sklearn.svm import SVC
import pickle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from IPython.display import Audio
from scipy.io import wavfile
import scipy
import soundfile as sf
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report
from pydub import AudioSegment
import random

import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from sklearn.metrics import classification_report, confusion_matrix
from pydub import AudioSegment


In [4]:
from pydub import AudioSegment
import os
import math
import random

from pydub.exceptions import CouldntDecodeError

def scan_audio_files(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(('.wav', '.mp3')):
                path = os.path.join(root, file)
                try:
                    _ = AudioSegment.from_file(path)
                except CouldntDecodeError:
                    print(f"🧨 Bad file: {path}")


def split_audio_file(input_file, all_clips, file_format):
    try:
        audio = AudioSegment.from_file(input_file, format=file_format)
        clip_duration_ms = 1000  # 1 second
        num_clips = math.ceil(len(audio) / clip_duration_ms)

        for start in range(0, len(audio), clip_duration_ms):
            clip = audio[start:start + clip_duration_ms]
            all_clips.append((clip, input_file, file_format))
    except Exception as e:
        print(f"❌ Could not process {input_file}: {e}")

def distribute_clips(all_clips, train_dir, test_dir):
    # Shuffle to randomize clips before splitting
    random.shuffle(all_clips)

    # Split clips 70% train, 30% test
    num_train = int(len(all_clips) * 0.7)
    train_clips = all_clips[:num_train]
    test_clips = all_clips[num_train:]

    # Save train clips
    for i, (clip, input_file, file_format) in enumerate(train_clips):
        relative_path = os.path.relpath(input_file, start=os.path.commonpath([input_file, train_dir]))
        save_dir = os.path.join(train_dir, os.path.dirname(relative_path))
        os.makedirs(save_dir, exist_ok=True)
        output_file = os.path.join(save_dir, f"{os.path.splitext(os.path.basename(input_file))[0]}_{i}.{file_format}")
        clip.export(output_file, format=file_format)
        print(f"Saved {output_file}")

    # Save test clips
    for i, (clip, input_file, file_format) in enumerate(test_clips):
        relative_path = os.path.relpath(input_file, start=os.path.commonpath([input_file, test_dir]))
        save_dir = os.path.join(test_dir, os.path.dirname(relative_path))
        os.makedirs(save_dir, exist_ok=True)
        output_file = os.path.join(save_dir, f"{os.path.splitext(os.path.basename(input_file))[0]}_{i + num_train}.{file_format}")
        clip.export(output_file, format=file_format)
        print(f"Saved {output_file}")



In [5]:
def process_directory(source_directory, train_directory, test_directory):
    all_clips = []
    for root, dirs, files in os.walk(source_directory):
        for file in files:
            if file.endswith(('.wav', '.mp3')):
                input_file_path = os.path.join(root, file)
                file_format = file.split('.')[-1]
                split_audio_file(input_file_path, all_clips, file_format)

    # Once all clips are generated, distribute them into train and test directories
    distribute_clips(all_clips, train_directory, test_directory)



In [6]:
# Directory setup
base_dir = '/content/drive/MyDrive/ASE_Dataset_Folder'
sound_dir = os.path.join(base_dir, 'Sounds')
sound_1_sec_dir = os.path.join(base_dir, 'Sounds_1_Sec')

In [ ]:
# Directory setup
base_dir = '/content/drive/MyDrive/ASE_Dataset_Folder'
sound_dir = os.path.join(base_dir, 'Sounds')
sound_1_sec_dir = os.path.join(base_dir, 'Sounds_1_Sec')

train_dir = os.path.join(sound_1_sec_dir, 'Train')
test_dir = os.path.join(sound_1_sec_dir, 'Test')

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

process_directory(sound_dir, train_dir, test_dir)


Saved /content/drive/MyDrive/ASE_Dataset_Folder/Sounds_1_Sec/Train/Sounds/Explosion/bomb-explosion-type-01-265510_0.mp3
Saved /content/drive/MyDrive/ASE_Dataset_Folder/Sounds_1_Sec/Train/Sounds/Explosion/explosion-large-129051_1.mp3
Saved /content/drive/MyDrive/ASE_Dataset_Folder/Sounds_1_Sec/Train/Sounds/Emergency_alarm/police-siren-21498_2.mp3
Saved /content/drive/MyDrive/ASE_Dataset_Folder/Sounds_1_Sec/Train/Sounds/Emergency_alarm/siren-7158_3.mp3
Saved /content/drive/MyDrive/ASE_Dataset_Folder/Sounds_1_Sec/Train/Sounds/Emergency_alarm/school-fire-alarm-loud-beepflac-14807_4.mp3
Saved /content/drive/MyDrive/ASE_Dataset_Folder/Sounds_1_Sec/Train/Sounds/Explosion/explosion-91004_5.mp3
Saved /content/drive/MyDrive/ASE_Dataset_Folder/Sounds_1_Sec/Train/Sounds/Emergency_alarm/thailand-eas-alarm-2006-266492_6.mp3
Saved /content/drive/MyDrive/ASE_Dataset_Folder/Sounds_1_Sec/Train/Sounds/Emergency_alarm/siren-blips-30816_7.mp3
Saved /content/drive/MyDrive/ASE_Dataset_Folder/Sounds_1_Sec/Tra

In [7]:
def count_files_in_folders(directory):
    """Counts files in each subfolder of the given directory."""
    for root, dirs, files in os.walk(directory):
        for folder in dirs:
            folder_path = os.path.join(root, folder)
            file_count = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])
            print(f"Folder: {folder_path}, File Count: {file_count}")



count_files_in_folders(sound_1_sec_dir)

Folder: /content/drive/MyDrive/ASE_Dataset_Folder/Sounds_1_Sec/Train, File Count: 0
Folder: /content/drive/MyDrive/ASE_Dataset_Folder/Sounds_1_Sec/Test, File Count: 0
Folder: /content/drive/MyDrive/ASE_Dataset_Folder/Sounds_1_Sec/Train/Sounds, File Count: 0
Folder: /content/drive/MyDrive/ASE_Dataset_Folder/Sounds_1_Sec/Train/Sounds/Explosion, File Count: 159
Folder: /content/drive/MyDrive/ASE_Dataset_Folder/Sounds_1_Sec/Train/Sounds/Emergency_alarm, File Count: 299
Folder: /content/drive/MyDrive/ASE_Dataset_Folder/Sounds_1_Sec/Train/Sounds/Gun_Shots, File Count: 93
Folder: /content/drive/MyDrive/ASE_Dataset_Folder/Sounds_1_Sec/Train/Sounds/glass_breaking, File Count: 140
Folder: /content/drive/MyDrive/ASE_Dataset_Folder/Sounds_1_Sec/Train/Sounds/thunderstorm, File Count: 142
Folder: /content/drive/MyDrive/ASE_Dataset_Folder/Sounds_1_Sec/Train/Sounds/dog, File Count: 144
Folder: /content/drive/MyDrive/ASE_Dataset_Folder/Sounds_1_Sec/Train/Sounds/wind, File Count: 134
Folder: /content/dr

**#For Specific Folders**

In [8]:
def process_directory(source_directory, train_directory, test_directory, target_folders):
    all_clips = []
    for root, dirs, files in os.walk(source_directory):
        # Check if the current directory's base name is in the target_folders list
        if os.path.basename(root) in target_folders:
            for file in files:
                if file.endswith(('.wav', '.mp3')):
                    input_file_path = os.path.join(root, file)
                    file_format = file.split('.')[-1]
                    split_audio_file(input_file_path, all_clips, file_format)

    # Once all clips are generated, distribute them into train and test directories
    distribute_clips(all_clips, train_directory, test_directory)

# Directory setup
base_dir = '/content/drive/MyDrive/ASE_Dataset_Folder'
sound_dir = os.path.join(base_dir, 'Sounds')
sound_1_sec_dir = os.path.join(base_dir, 'Sounds_1_Sec')

train_dir = os.path.join(sound_1_sec_dir, 'Train')
test_dir = os.path.join(sound_1_sec_dir, 'Test')

# Only process these specific folders
target_folders = ['Wildfire']

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

scan_audio_files(sound_dir)

process_directory(sound_dir, train_dir, test_dir, target_folders)


❌ Could not process /content/drive/MyDrive/ASE_Dataset_Folder/Sounds/Wildfire/Forest Fire16.mp3: Decoding failed. ffmpeg returned error code: 1

Output from ffmpeg/avlib:

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsn

In [9]:
def count_files_in_folders(directory):
    """Counts files in each subfolder of the given directory."""
    for root, dirs, files in os.walk(directory):
        for folder in dirs:
            folder_path = os.path.join(root, folder)
            file_count = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])
            print(f"Folder: {folder_path}, File Count: {file_count}")



count_files_in_folders(sound_1_sec_dir)

Folder: /content/drive/MyDrive/ASE_Dataset_Folder/Sounds_1_Sec/Train, File Count: 0
Folder: /content/drive/MyDrive/ASE_Dataset_Folder/Sounds_1_Sec/Test, File Count: 0
Folder: /content/drive/MyDrive/ASE_Dataset_Folder/Sounds_1_Sec/Train/Sounds, File Count: 0
Folder: /content/drive/MyDrive/ASE_Dataset_Folder/Sounds_1_Sec/Train/Sounds/Explosion, File Count: 159
Folder: /content/drive/MyDrive/ASE_Dataset_Folder/Sounds_1_Sec/Train/Sounds/Emergency_alarm, File Count: 299
Folder: /content/drive/MyDrive/ASE_Dataset_Folder/Sounds_1_Sec/Train/Sounds/Gun_Shots, File Count: 93
Folder: /content/drive/MyDrive/ASE_Dataset_Folder/Sounds_1_Sec/Train/Sounds/glass_breaking, File Count: 140
Folder: /content/drive/MyDrive/ASE_Dataset_Folder/Sounds_1_Sec/Train/Sounds/thunderstorm, File Count: 142
Folder: /content/drive/MyDrive/ASE_Dataset_Folder/Sounds_1_Sec/Train/Sounds/dog, File Count: 144
Folder: /content/drive/MyDrive/ASE_Dataset_Folder/Sounds_1_Sec/Train/Sounds/wind, File Count: 134
Folder: /content/dr

In [10]:
import os

def count_total_files(directory):
    """Counts total files in all subfolders of the given directory."""
    total_count = 0
    for root, _, files in os.walk(directory):
        count = len([f for f in files if os.path.isfile(os.path.join(root, f))])
        total_count += count
    return total_count

# Define paths
train_dir = os.path.join(sound_1_sec_dir, 'Train')
test_dir = os.path.join(sound_1_sec_dir, 'Test')

# Count files
train_count = count_total_files(train_dir)
test_count = count_total_files(test_dir)
total = train_count + test_count

print(f"🟢 Train Files: {train_count}")
print(f"🔵 Test Files: {test_count}")
print(f"🟡 Total Files: {total}")


🟢 Train Files: 4056
🔵 Test Files: 1741
🟡 Total Files: 5797
